In [1]:
from PyQt5.QtWidgets import QFileDialog, QApplication

def gui_fname(dir=None):
    """
    Select a file via a dialog and return the file name.
    """
    if dir is None: 
        dir ="./"

    app = QApplication([dir])
    fname = QFileDialog.getExistingDirectory(None, "Select a folder...", 
            dir)
    if isinstance(fname, tuple):
        return fname[0]
    else: 
        return str(fname)
    
def load_ring_data_df(dirname, filename):
    file = ospath.join(dirname, filename)

    try: 
        df = pd.read_pickle(file)
    except FileNotFoundError:
        print("No results of previously analyzed datasets were detected.")
        return None
    else: 
        print("Results of previously analyzed datasets were detected.")
        return df
    
    
def load_data(path):

    try:
        locs, info = io.load_locs(path)
    except io.NoMetadataFileError:
        return None, None, None
    
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")
  
    if hasattr(locs, "x_pick_rot"):

        # convert px to nm
        locs.x *= pixelsize
        locs.y *= pixelsize
        locs.x_pick_rot *= pixelsize
        locs.y_pick_rot *= pixelsize
        
        return locs, info, pixelsize
    else:
        return None, None, None
    
def save_ring_locs(locs, info, path, file_id, pick, id, link=False):
    
    if link:
        ending = "_link.hdf5"
    else:
        ending = ".hdf5"
       
    locs.x /= 130
    locs.y /= 130
    
    locs_name = "file_{}_pick_{}_ring_{}{}".format(file_id, pick, id, ending)
    locs_path = os.path.join(path,"ring_locs")
    locs_path_name = os.path.join(locs_path, locs_name)
    
    if not os.path.isdir(locs_path):
        os.makedirs(locs_path)
    
    io.save_locs(locs_path_name, locs, info)

In [2]:
path = gui_fname()
